In [12]:
VERSION = 3.1

In [13]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.14.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
from tensorflow.python.client import device_lib

local_devices = device_lib.list_local_devices()
gpus = [device.name for device in local_devices if device.device_type == 'GPU']
print("Available GPUs:", gpus)

Available GPUs: ['/device:GPU:0']


2023-11-14 15:41:33.065312: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-14 15:41:33.065441: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-14 15:41:33.065486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [15]:
import pandas as pd
import numpy as np
from tensorflow import keras
from pyswarm import pso

In [16]:
data = pd.read_csv('training.csv', header=None)
data = data * (10**3)
data.head()

,0,1,2,3,4
0,-145.0,-145.0,-145.0,-145.0,-145.0
1,-145.0,-145.0,-145.0,-145.0,-145.0
2,-145.0,-145.0,-135.0,-145.0,-120.0
3,-135.0,-145.0,-160.0,-155.0,-150.0
4,-160.0,-155.0,-175.0,-180.0,-160.0


In [17]:
from tensorflow.python.client import device_lib

local_devices = device_lib.list_local_devices()
gpus = [device.name for device in local_devices if device.device_type == 'GPU']
if gpus:
    print("Keras menggunakan GPU:", gpus)
else:
    print("Keras menggunakan CPU")

Keras menggunakan GPU: ['/device:GPU:0']


2023-11-14 15:41:33.094255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-14 15:41:33.094377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-14 15:41:33.094421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [18]:
# Membuat dataset TensorFlow dari data Anda
# Menggunakan data yang telah Anda baca dari berkas CSV
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

print("x", X)
print("y", y)
print("min", X.min(), "max", X.max())

x [[-145. -145. -145. -145.]
 [-145. -145. -145. -145.]
 [-145. -145. -135. -145.]
 ...
 [-210. -210. -205. -210.]
 [-205. -210. -225. -220.]
 [-225. -220. -210. -205.]]
y [-145. -145. -120. ... -205. -215. -235.]
min -3095.0 max 2495.0


In [19]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

# Menambahkan metode prefetch() ke dataset
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [20]:
tf.debugging.set_log_device_placement(True)
index = 0

# Batasan parameter untuk PSO
num_input_units = data.shape[1] - 1
num_hidden_units = 10
num_output_units = 1

num_weights_layer1 = num_input_units * num_hidden_units
num_bias_layer1 = num_hidden_units
num_weights_layer2 = num_hidden_units * num_output_units
num_bias_layer2 = num_output_units

# Fungsi objektif untuk PSO (menggunakan MSE sebagai objektif)
def objective_function(params):
    global index
    print(f'Proses ke-{index}')

    # Membangun model jaringan saraf tiruan
    model = keras.Sequential([
        keras.layers.Dense(10, input_dim=data.shape[1] - 1, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    
    # Set parameter berdasarkan hasil optimasi PSO    
    weights_layer1 = params[:num_weights_layer1].reshape((num_input_units, num_hidden_units))
    bias_layer1 = params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
    weights_layer2 = params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((num_hidden_units, num_output_units))
    bias_layer2 = params[-num_bias_layer2:]
    
    model.layers[0].set_weights([weights_layer1, bias_layer1])
    model.layers[1].set_weights([weights_layer2, bias_layer2])
    
    # Kompilasi model
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    # Latih model
    print(f'Fit ke-{index}')
    model.fit(dataset, epochs=100, verbose=0)
    print(f'Fit ke-{index} End')
    
    print('Predict: ', model.predict([[-145., - 145., - 145., - 145.]]))

    # Evaluasi model dan mengembalikan nilai yang ingin dioptimasi
    loss = model.evaluate(X, y, verbose=1)
    print(f'Loss ke-{index}', loss)
    index += 1
    return loss


num_parameters = num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2
lb = [-1.0] * num_parameters  # Batas bawah parameter
ub = [1.0] * num_parameters   # Batas atas parameter

# Jalankan PSO untuk mengoptimasi parameter jaringan
best_params, _ = pso(objective_function, lb, ub, swarmsize=10, maxiter=50)
best_params

Proses ke-0
Fit ke-0
Fit ke-0 End
1/1 [==============================] - 0s 31ms/step
Predict:  [[-144.0116]]
1724/1724 [==============================] - 1s 593us/step - loss: 97.6838
Loss ke-0 97.683837890625
Proses ke-1
Fit ke-1
Fit ke-1 End
1/1 [==============================] - 0s 30ms/step
Predict:  [[-144.60645]]
1724/1724 [==============================] - 1s 585us/step - loss: 80.4762
Loss ke-1 80.47621154785156
Proses ke-2
Fit ke-2
Fit ke-2 End
1/1 [==============================] - 0s 30ms/step
Predict:  [[-144.2841]]
1724/1724 [==============================] - 1s 581us/step - loss: 93.7068
Loss ke-2 93.706787109375
Proses ke-3
Fit ke-3
Fit ke-3 End
1/1 [==============================] - 0s 29ms/step
Predict:  [[-144.51273]]
1724/1724 [==============================] - 1s 574us/step - loss: 83.8379
Loss ke-3 83.83792114257812
Proses ke-4
Fit ke-4
Fit ke-4 End
1/1 [==============================] - 0s 29ms/step
Predict:  [[-144.6406]]
1724/1724 [=============================

array([ 0.23119709, -0.14332756,  0.29198279,  0.48255522,  0.64122843,
        0.33432868, -0.40972599,  0.60236862,  0.55619598, -0.03009837,
       -0.39509408, -0.83735936,  0.19259505,  0.71971256, -0.42746288,
        0.40280159,  0.37806378,  0.54209209, -0.34170424,  0.45023116,
       -0.95268347, -0.19301141,  0.44147429,  0.68514488,  0.96822297,
        0.44990865,  0.21753976, -0.26990627, -0.25750184, -0.39227759,
        0.24887333,  0.09226722,  0.36019504,  0.51274085, -0.89012132,
        0.19456852,  0.71004253,  0.06572839, -0.18266101, -0.4893083 ,
        0.05871857,  0.00619018,  0.37274036, -0.76595252, -0.76531339,
       -0.10227559,  0.62537872, -0.07779787, -0.17152452, -0.06653802,
        0.22810615, -0.8526836 ,  0.48395477, -0.21296287, -0.47793783,
       -0.05304364,  0.08881288,  0.94322136, -0.50090546, -0.99652957,
        0.20078839])

In [21]:
# Bangun model jaringan saraf tiruan dengan parameter terbaik
best_model = keras.Sequential([
    keras.layers.Dense(10, input_dim=num_input_units, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])
weights_layer1 = best_params[:num_weights_layer1].reshape((num_input_units, num_hidden_units))
bias_layer1 = best_params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
weights_layer2 = best_params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((num_hidden_units, num_output_units))
bias_layer2 = best_params[-num_bias_layer2:]


best_model.layers[0].set_weights([weights_layer1, bias_layer1])
best_model.layers[1].set_weights([weights_layer2, bias_layer2])

In [22]:
best_model.predict([[-145., - 145., - 145., - 145.]])

1/1 [==============================] - 0s 36ms/step


array([[-187.7156]], dtype=float32)

In [23]:
# Compile model dengan optimizer dan loss function
best_model.compile(optimizer='adam', loss='mean_squared_error')

# Simpan model terbaik ke dalam berkas
best_model.save(f"ann_pso_v{VERSION}.keras")

In [24]:
model = tf.keras.models.load_model(f"ann_pso_v{VERSION}.keras")

In [25]:
model.predict([[-145., - 145., - 145., - 145.]])

1/1 [==============================] - 0s 36ms/step


array([[-187.7156]], dtype=float32)